In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('fitness_exercises_500.csv')

In [3]:
df.head()

,exercise_name,type_of_activity,type_of_equipment,body_part,type,muscle_groups_activated,instruction
0,Push-Up Hold,mobility,dip belt,lower,stretch,"glutes, quads, hamstrings",Push-Up Hold: Setup — Use dip belt. Focus on s...
1,Explosive Lateral Raise Pulse,cardio,barbells,full body,hold,"back, chest, legs",Explosive Lateral Raise Pulse: Setup — Use bar...
2,Rotational Jumping Jack Iso,warm-up,barbells,upper,push,"biceps, forearms, chest",Rotational Jumping Jack Iso: Setup — Use barbe...
3,Wide-Grip Running,strength,kettlebells,upper,pull,"deltoids, triceps, forearms",Wide-Grip Running: Setup — Use kettlebells. Fo...
4,Decline Dips,cardio,barbells,core,stretch,"abs, lower back, obliques",Decline Dips: Setup — Use barbells. Focus on s...


In [4]:
import minsearch

ModuleNotFoundError: No module named 'minsearch'